# Analiza gibanja - zaporedje slik

## Importi

In [2]:
%matplotlib qtagg

In [1]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage.registration as skreg
import pathlib
import os

## Branje videoposnetkov

In [37]:
# nalaganje in predvajanje z matplotlib
pot_do_posnetka = pathlib.Path('./data/vid1/')
slike_poti = pot_do_posnetka.glob('frame*.jpg')
# te posnetke lahko uredimo po abecednem vrstnem redu, da dobimo ustrezno video sekvenco
slike_poti = sorted(slike_poti)
plt.figure()
video_seq = []
for pot in slike_poti:
    slika = plt.imread(pot)
    video_seq.append(slika)

    plt.clf()
    plt.imshow(slika)
    plt.draw()
    plt.waitforbuttonpress(0.01)
    
plt.close('all')
video = np.array(video_seq)

In [3]:
# nalaganje in predvajanje z matplotlib
pot_do_posnetka = pathlib.Path('./data/porocilo_drugi_video/')
slike_poti = pot_do_posnetka.glob('frame*.jpg')
# te posnetke lahko uredimo po abecednem vrstnem redu, da dobimo ustrezno video sekvenco
slike_poti = sorted(slike_poti)
plt.figure()
video_seq = []
for pot in slike_poti:
    slika = plt.imread(pot)
    video_seq.append(slika)

    plt.clf()
    plt.imshow(slika)
    plt.draw()
    plt.waitforbuttonpress(0.01)
    
plt.close('all')
video = np.array(video_seq)

In [20]:
video.shape, video.dtype

((121, 180, 180, 3), dtype('uint8'))

In [35]:
# če bi želeli narediti biti izredno računsko in prostorsko učinkoviti, bi večino časa uporabljali celoštevilske tipe kot je uint8
# vendar moramo potem biti izredno previdni katere izračune opravljamo in pogosto pretvarjati med tipi
# dosti lažje že od začetka pretvoriti v np.float32 in vrednosti pretvoriti na interval [0,1]

# ZGOLJ CE MISLIS DELAT Z OPENCV METODO ZA ODSTRANJEVANJE OZADJA!!!
video = video.astype(np.float32)/255.
print(video.dtype)

float32


## Odstranjevanje ozadja

### Preprosta ocena ozadja s povprečenjem

In [ ]:
slika_bg = video.mean(0)

n=120
slika = video[n]
slika_diff = np.abs(slika-slika_bg).mean(2) # povprecje po barvah, ce imamo barve

plt.figure()
plt.subplot(221)
plt.imshow(slika_bg)
plt.title('ozadje')
plt.axis('off')
plt.subplot(222)
plt.imshow(slika)
plt.title(f'slika {n}')
plt.axis('off')
plt.subplot(223)
plt.imshow(slika_diff)
plt.title('slika razlik')
plt.axis('off')
plt.subplot(224)
plt.hist(slika_diff.ravel(), bins=100, log=True)
plt.title('histogram slike razlik')

In [25]:
prag = 0.2

plt.figure()
for n in range(video.shape[0]):
    slika = video[n]
    slika_diff = np.abs(slika-slika_bg).mean(2) 
    slika_motion_seg = slika_diff>prag

    plt.clf()
    plt.imshow(slika_motion_seg)
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')

#### Končna rešitev uporabljen v nalogi

In [21]:
# 1. Izračun povprečne slike ozadja
slika_bg = video.mean(axis=0)  # Povprečje po časovni dimenziji

# 2. Seznam za shranjevanje mask segmentacije
segmentation_masks = []

# 3. Parametri za segmentacijo
threshold_value = 50  # Prag za segmentacijo
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Jedro za morfološke operacije

plt.figure()
for n in range(video.shape[0]):
    # Trenutna slika in razlika od ozadja
    slika = video[n]
    slika_diff = np.abs(slika - slika_bg).mean(axis=2)  # Povprečje po barvnih kanalih

    # 4. Pragovna segmentacija
    _, segmented = cv2.threshold(slika_diff.astype(np.uint8), threshold_value, 255, cv2.THRESH_BINARY)

    # 5. Morfološke operacije za izboljšanje maske
    segmented = cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, kernel)  # Zapiranje za odpravo lukenj
    segmented = cv2.morphologyEx(segmented, cv2.MORPH_OPEN, kernel)   # Odpiranje za odstranitev šuma

    # Shrani masko
    segmentation_masks.append(segmented)

    # Vizualizacija trenutne slike, ozadja, razlike in maske
    plt.clf()
    plt.subplot(2, 2, 1)
    plt.imshow(slika_bg.astype(np.uint8))
    plt.title('Ozadje')
    plt.axis('off')
    plt.subplot(2, 2, 2)
    plt.imshow(slika.astype(np.uint8))
    plt.title(f'Slika {n}')
    plt.axis('off')
    plt.subplot(2, 2, 3)
    plt.imshow(slika_diff, cmap='gray')
    plt.title('Razlika od ozadja')
    plt.axis('off')
    plt.subplot(2, 2, 4)
    plt.imshow(segmented, cmap='gray')
    plt.title('Segmentirana maska')
    plt.axis('off')
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')

print("sucess")
# 6. Pretvori seznam mask v numpy array, če je potrebno
segmentation_masks_array = np.array(segmentation_masks, dtype=np.uint8)


sucess


In [50]:
import pathlib

output_folder = './data/segmentation_masks_hitre/'
pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

for i, mask in enumerate(segmentation_masks):
    output_path = f"{output_folder}/mask_{i:04d}.png"
    cv2.imwrite(output_path, mask)

## Uporaba OpenCV metode

### MOG2 method - ta metoda ni bila uporabljena v končni rešitvi

In [23]:
bg_sub_obj = cv2.createBackgroundSubtractorMOG2()
segmentation_masks = []

plt.figure()
for n in range(video.shape[0]):
    slika = np.uint8(video[n]*255)
    # opencv je malo razvajen, hoče imeti zelo specifične tipe, da deluje pravilno
    slika_motion_seg = bg_sub_obj.apply(slika)
    slika_motion_bg = bg_sub_obj.getBackgroundImage()

    # Segmentacija z adaptivnim pragom
    _, segmented = cv2.threshold(slika_motion_seg, 50, 255, cv2.THRESH_BINARY)
    
    # Morfološke operacije za izboljšanje
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    segmented = cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, kernel)  # Zapiranje

    # Shrani masko za analizo ali nadaljnjo obdelavo
    segmentation_masks.append(segmented)
    
    plt.clf()
    plt.subplot(1,3,1)
    plt.imshow(slika)
    plt.title('posnetek')
    plt.subplot(1,3,2)
    plt.imshow(slika_motion_bg)
    plt.title('slika ozadja')
    plt.subplot(1,3,3)
    plt.imshow(slika_motion_seg)
    plt.title('segmenti gibanja')
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')

## Optični pretok v OpenCV

### Iskanje kontur

In [22]:
# Zbirka za konture na vsaki sliki
filtered_contours_per_frame = []

plt.figure()
for n, mask in enumerate(segmentation_masks):
    # Najdi konture
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtriraj konture na podlagi površine
    min_area = 160  # Minimalna površina gibajočega elementa
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    filtered_contours_per_frame.append(filtered_contours)

    # Vizualizacija
    mask_with_contours = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  # Pretvori masko v RGB za risanje kontur
    cv2.drawContours(mask_with_contours, filtered_contours, -1, (0, 255, 0), 2)  # Zeleno za konture
    
    plt.clf()
    plt.subplot(121)
    plt.imshow(mask, cmap='gray')
    plt.title('Segmentirana maska')
    plt.subplot(122)
    plt.imshow(mask_with_contours)
    plt.title('Konture')
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')


### Iskanje prekrivajočih kontur

In [23]:
def rect_overlap(rect1, rect2):
    """Preveri, ali se dva pravokotnika prekrivata."""
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    return not (x1 + w1 < x2 or x2 + w2 < x1 or y1 + h1 < y2 or y2 + h2 < y1)

# Seznam za shranjevanje povezanih gibajočih elementov
moving_elements = []

# Prvi okvir
prev_contours = filtered_contours_per_frame[0]

# Določitev gibajočih elementov skozi sličice
for i in range(1, len(filtered_contours_per_frame)):
    curr_contours = filtered_contours_per_frame[i]
    
    # Seznam za trenutno sličico
    curr_elements = []
    
    for curr_cnt in curr_contours:
        curr_rect = cv2.boundingRect(curr_cnt)
        matched = False
        
        for prev_cnt in prev_contours:
            prev_rect = cv2.boundingRect(prev_cnt)
            
            if rect_overlap(curr_rect, prev_rect):
                # Če se prekrivata, poveži kot isti gibajoči element
                matched = True
                break
        
        # Dodaj trenutni element (lahko ga kasneje označiš kot nov, če ni povezave)
        curr_elements.append(curr_cnt)
    
    # Posodobi konture za naslednji korak
    prev_contours = curr_contours

    # Shrani trenutne elemente
    moving_elements.append(curr_elements)

print("Gibajoči elementi so bili uspešno povezani!")

Gibajoči elementi so bili uspešno povezani!


In [24]:
plt.figure()
for i, elements in enumerate(moving_elements):
    mask_with_contours = np.zeros_like(segmentation_masks[i])
    for cnt in elements:
        cv2.drawContours(mask_with_contours, [cnt], -1, 255, -1)  # Bela za gibajoče elemente
    
    plt.clf()
    plt.imshow(mask_with_contours, cmap='gray')
    plt.title(f'Gibajoči elementi v sličici {i}')
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')


### Pridobivanje analiz

In [25]:
# željene analize:
analize = []
# Inicializacija optičnega pretoka za vsak par sličic
flow_maps = []
for n in range(video.shape[0] - 1):
    slika_0 = (video[n] * 255).mean(2).astype(np.uint8)
    slika_1 = (video[n + 1] * 255).mean(2).astype(np.uint8)
    flow = cv2.calcOpticalFlowFarneback(slika_0, slika_1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_maps.append(flow)

# Povprečen optični pretok za gibajoče elemente
plt.figure()
for i, (elements, flow) in enumerate(zip(moving_elements, flow_maps)):
    # Inicializiraj barvno sliko (HSV format)
    h, w = flow.shape[:2]
    hsv_img = np.zeros((h, w, 3), dtype=np.float32)
    hsv_img[..., 1] = 1  # Saturation na maksimum

    for cnt in elements:
        # Ustvari masko za trenutno konturo
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.drawContours(mask, [cnt], -1, 255, -1)

        # Povprečni dx, dy znotraj konture
        dx_mean = np.mean(flow[..., 0][mask == 255])
        dy_mean = np.mean(flow[..., 1][mask == 255])

        # Pretvorba v amplitudo in smer
        amplitude = np.sqrt(dx_mean**2 + dy_mean**2)
        direction = np.arctan2(dy_mean, dx_mean)  # Smer v radianih

        # Pretvorba v barvno kodacijo (Hue in Value)
        hue = (direction * 180 / np.pi / 2) % 180  # Pretvori radiane v stopinje (Hue)
        value = np.clip(amplitude / amplitude.max(), 0, 1)  # Normalizacija amplitude

        # Barvna kodacija v območju konture
        hsv_img[..., 0][mask == 255] = hue  # Hue
        hsv_img[..., 2][mask == 255] = value  # Value

    # Pretvorba iz HSV v RGB za prikaz
    rgb_img = cv2.cvtColor((hsv_img * 255).astype(np.uint8), cv2.COLOR_HSV2RGB)
    analize.append(rgb_img)

    # Vizualizacija
    plt.clf()
    plt.imshow(rgb_img)
    plt.title(f'Amplituda in smer gibanja v sličici {i}')
    plt.axis('off')
    plt.draw()
    plt.waitforbuttonpress(0.01)

plt.close('all')

In [21]:
import os

# Ustvari mapo, če ne obstaja
output_dir = './data/analize'
os.makedirs(output_dir, exist_ok=True)

# Shrani maske
for i, analiza in enumerate(analize):
    # Definiraj ime datoteke
    analiza_path = os.path.join(output_dir, f'analiza_{i:03d}.png')
    
    # Shrani masko kot sliko
    cv2.imwrite(analiza_path, analiza)

print(f"Vse analize so bile shranjene v mapo {output_dir}")

Vse analize so bile shranjene v mapo ./data/analize


## KREACIJA FUNKCIJE:

##### Vsi videoposnetki (podani v primeru, in lastni) so bili podani v spodnjo funkcijo in v končno združeno funkcijo.

In [42]:
test_video = pathlib.Path('./data/vid1/')
slike_poti = test_video.glob('frame*.jpg')

In [5]:
test_video_3 = pathlib.Path('./data/porocilo_video/')
slike_poti_3 = test_video_3.glob('frame*.jpg')

In [19]:
test_video_4 = pathlib.Path('./data/porocilo_drugi_video//')
slike_poti_4 = test_video_4.glob('frame*.jpg')

In [10]:
test_synth = pathlib.Path('./data/synth_vid1/')
slike_poti_synth = test_synth.glob('frame*.jpg')

In [28]:
test_synth_2 = pathlib.Path('./data/synth_vid2/')
slike_poti_synth_2 = test_synth_2.glob('frame*.jpg')

In [12]:
def rect_overlap(rect1, rect2):
    """Preveri, ali se dva pravokotnika prekrivata."""
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    return not (x1 + w1 < x2 or x2 + w2 < x1 or y1 + h1 < y2 or y2 + h2 < y1)

In [13]:
def analiziraj_opticni_pretok(video: np.ndarray) -> list[np.ndarray]:
    #pot_do_posnetka = pathlib.Path('./data/vid1/')
    #slike_poti = video.glob('frame*.jpg')
    # te posnetke lahko uredimo po abecednem vrstnem redu, da dobimo ustrezno video sekvenco
    slike_poti = sorted(video)
    video_seq = []
    for pot in slike_poti:
        slika = plt.imread(pot)
        video_seq.append(slika)
    video = np.array(video_seq)


    # 1. Izračun povprečne slike ozadja
    slika_bg = video.mean(axis=0)  # Povprečje po časovni dimenziji
    
    # 2. Seznam za shranjevanje mask segmentacije
    segmentation_masks = []
    
    # 3. Parametri za segmentacijo
    threshold_value = 50  # Prag za segmentacijo
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Jedro za morfološke operacije

    for n in range(video.shape[0]):
        # Trenutna slika in razlika od ozadja
        slika = video[n]
        slika_diff = np.abs(slika - slika_bg).mean(axis=2)  # Povprečje po barvnih kanalih
    
        # 4. Pragovna segmentacija
        _, segmented = cv2.threshold(slika_diff.astype(np.uint8), threshold_value, 255, cv2.THRESH_BINARY)
    
        # 5. Morfološke operacije za izboljšanje maske
        segmented = cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, kernel)  # Zapiranje za odpravo lukenj
        segmented = cv2.morphologyEx(segmented, cv2.MORPH_OPEN, kernel)   # Odpiranje za odstranitev šuma
    
        # Shrani masko
        segmentation_masks.append(segmented)

    # Zbirka za konture na vsaki sliki
    filtered_contours_per_frame = []
    
    for n, mask in enumerate(segmentation_masks):
        # Najdi konture
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        # Filtriraj konture na podlagi površine
        min_area = 160  # Minimalna površina gibajočega elementa
        filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
        filtered_contours_per_frame.append(filtered_contours)

    # Seznam za shranjevanje povezanih gibajočih elementov
    moving_elements = []
    
    # Prvi okvir
    prev_contours = filtered_contours_per_frame[0]
    
    # Določitev gibajočih elementov skozi sličice
    for i in range(1, len(filtered_contours_per_frame)):
        curr_contours = filtered_contours_per_frame[i]
        
        # Seznam za trenutno sličico
        curr_elements = []
        
        for curr_cnt in curr_contours:
            curr_rect = cv2.boundingRect(curr_cnt)
            matched = False
            
            for prev_cnt in prev_contours:
                prev_rect = cv2.boundingRect(prev_cnt)
                
                if rect_overlap(curr_rect, prev_rect):
                    # Če se prekrivata, poveži kot isti gibajoči element
                    matched = True
                    break
            
            # Dodaj trenutni element (lahko ga kasneje označiš kot nov, če ni povezave)
            curr_elements.append(curr_cnt)
        
        # Posodobi konture za naslednji korak
        prev_contours = curr_contours
    
        # Shrani trenutne elemente
        moving_elements.append(curr_elements)
    
    print("Gibajoči elementi so bili uspešno povezani!")

    # željene analize:
    analize = []
    # Inicializacija optičnega pretoka za vsak par sličic
    flow_maps = []
    for n in range(video.shape[0] - 1):
        slika_0 = (video[n] * 255).mean(2).astype(np.uint8)
        slika_1 = (video[n + 1] * 255).mean(2).astype(np.uint8)
        flow = cv2.calcOpticalFlowFarneback(slika_0, slika_1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_maps.append(flow)
    
    # Povprečen optični pretok za gibajoče elemente
    for i, (elements, flow) in enumerate(zip(moving_elements, flow_maps)):
        # Inicializiraj barvno sliko (HSV format)
        h, w = flow.shape[:2]
        hsv_img = np.zeros((h, w, 3), dtype=np.float32)
        hsv_img[..., 1] = 1  # Saturation na maksimum
    
        for cnt in elements:
            # Ustvari masko za trenutno konturo
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.drawContours(mask, [cnt], -1, 255, -1)
    
            # Povprečni dx, dy znotraj konture
            dx_mean = np.mean(flow[..., 0][mask == 255])
            dy_mean = np.mean(flow[..., 1][mask == 255])
    
            # Pretvorba v amplitudo in smer
            amplitude = np.sqrt(dx_mean**2 + dy_mean**2)
            direction = np.arctan2(dy_mean, dx_mean)  # Smer v radianih
    
            # Pretvorba v barvno kodacijo (Hue in Value)
            hue = (direction * 180 / np.pi / 2) % 180  # Pretvori radiane v stopinje (Hue)
            value = np.clip(amplitude / amplitude.max(), 0, 1)  # Normalizacija amplitude
    
            # Barvna kodacija v območju konture
            hsv_img[..., 0][mask == 255] = hue  # Hue
            hsv_img[..., 2][mask == 255] = value  # Value
    
        # Pretvorba iz HSV v RGB za prikaz
        rgb_img = cv2.cvtColor((hsv_img * 255).astype(np.uint8), cv2.COLOR_HSV2RGB)
        analize.append(rgb_img)

    return analize

In [43]:
dobljene_analize = analiziraj_opticni_pretok(slike_poti)

Gibajoči elementi so bili uspešno povezani!


In [63]:
dobljene_analize_2 = analiziraj_opticni_pretok(slike_poti_2)

Gibajoči elementi so bili uspešno povezani!


In [8]:
dobljene_analize_3 = analiziraj_opticni_pretok(slike_poti_3)

Gibajoči elementi so bili uspešno povezani!


In [20]:
dobljene_analize_4 = analiziraj_opticni_pretok(slike_poti_4)

Gibajoči elementi so bili uspešno povezani!


In [14]:
dobljene_analize_synth = analiziraj_opticni_pretok(slike_poti_synth)

Gibajoči elementi so bili uspešno povezani!


In [17]:
dobljene_analize_synth_2 = analiziraj_opticni_pretok(slike_poti_synth_2)

Gibajoči elementi so bili uspešno povezani!


In [30]:
# Ustvari mapo, če ne obstaja
output_dir = './data/analize_iz_fun_synth_2_druga'
os.makedirs(output_dir, exist_ok=True)

# Shrani maske
for i, analiza_fun in enumerate(dobljene_analize_synth_2_0):
    # Definiraj ime datoteke
    analiza_path = os.path.join(output_dir, f'analiza_fun_{i:03d}.png')
    
    # Shrani masko kot sliko
    cv2.imwrite(analiza_path, analiza_fun)

print(f"Vse analize funkcije so bile shranjene v mapo {output_dir}")

Vse analize funkcije so bile shranjene v mapo ./data/analize_iz_fun_synth_2_druga


# KONČNA SKUPNA FUNKCIJA

In [26]:
def analiziraj_opticni_pretok_2(video: np.ndarray) -> list[np.ndarray]:
    slike_poti = sorted(video)
    video_seq = []
    for pot in slike_poti:
        slika = plt.imread(pot)
        video_seq.append(slika)
    video = np.array(video_seq)

    # 1. Izračun povprečne slike ozadja
    slika_bg = video.mean(axis=0)  # Povprečje po časovni dimenziji

    # 2. Seznam za shranjevanje mask segmentacije
    segmentation_masks = []

    # 3. Parametri za segmentacijo
    threshold_value = 50  # Prag za segmentacijo
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Jedro za morfološke operacije

    for n in range(video.shape[0]):
        # Trenutna slika in razlika od ozadja
        slika = video[n]
        slika_diff = np.abs(slika - slika_bg).mean(axis=2)  # Povprečje po barvnih kanalih

        # 4. Pragovna segmentacija
        _, segmented = cv2.threshold(slika_diff.astype(np.uint8), threshold_value, 255, cv2.THRESH_BINARY)

        # 5. Morfološke operacije za izboljšanje maske
        segmented = cv2.morphologyEx(segmented, cv2.MORPH_CLOSE, kernel)  # Zapiranje za odpravo lukenj
        segmented = cv2.morphologyEx(segmented, cv2.MORPH_OPEN, kernel)  # Odpiranje za odstranitev šuma

        # Shrani masko
        segmentation_masks.append(segmented)

    # Zbirka za konture na vsaki sliki
    filtered_contours_per_frame = []

    for n, mask in enumerate(segmentation_masks):
        # Najdi konture
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filtriraj konture na podlagi površine
        min_area = 160  # Minimalna površina gibajočega elementa
        filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
        filtered_contours_per_frame.append(filtered_contours)

    # Seznam za shranjevanje povezanih gibajočih elementov
    moving_elements = []

    # Prvi okvir
    prev_contours = filtered_contours_per_frame[0]

    # Določitev gibajočih elementov skozi sličice
    for i in range(1, len(filtered_contours_per_frame)):
        curr_contours = filtered_contours_per_frame[i]

        # Seznam za trenutno sličico
        curr_elements = []

        for curr_cnt in curr_contours:
            curr_rect = cv2.boundingRect(curr_cnt)
            matched = False

            for prev_cnt in prev_contours:
                prev_rect = cv2.boundingRect(prev_cnt)

                # Preveri, ali se pravokotnika prekrivata
                if not (curr_rect[0] + curr_rect[2] < prev_rect[0] or
                        prev_rect[0] + prev_rect[2] < curr_rect[0] or
                        curr_rect[1] + curr_rect[3] < prev_rect[1] or
                        prev_rect[1] + prev_rect[3] < curr_rect[1]):
                    matched = True
                    break

            # Dodaj trenutni element (lahko ga kasneje označiš kot nov, če ni povezave)
            curr_elements.append(curr_cnt)

        # Posodobi konture za naslednji korak
        prev_contours = curr_contours

        # Shrani trenutne elemente
        moving_elements.append(curr_elements)

    print("Gibajoči elementi so bili uspešno povezani!")

    # Željene analize:
    analize = []
    # Inicializacija optičnega pretoka za vsak par sličic
    flow_maps = []
    for n in range(video.shape[0] - 1):
        slika_0 = (video[n] * 255).mean(2).astype(np.uint8)
        slika_1 = (video[n + 1] * 255).mean(2).astype(np.uint8)
        flow = cv2.calcOpticalFlowFarneback(slika_0, slika_1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_maps.append(flow)

    # Povprečen optični pretok za gibajoče elemente
    for i, (elements, flow) in enumerate(zip(moving_elements, flow_maps)):
        # Inicializiraj barvno sliko (HSV format)
        h, w = flow.shape[:2]
        hsv_img = np.zeros((h, w, 3), dtype=np.float32)
        hsv_img[..., 1] = 1  # Saturation na maksimum

        for cnt in elements:
            # Ustvari masko za trenutno konturo
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.drawContours(mask, [cnt], -1, 255, -1)

            # Povprečni dx, dy znotraj konture
            dx_mean = np.mean(flow[..., 0][mask == 255])
            dy_mean = np.mean(flow[..., 1][mask == 255])

            # Pretvorba v amplitudo in smer
            amplitude = np.sqrt(dx_mean ** 2 + dy_mean ** 2)
            direction = np.arctan2(dy_mean, dx_mean)  # Smer v radianih

            # Pretvorba v barvno kodacijo (Hue in Value)
            hue = (direction * 180 / np.pi / 2) % 180  # Pretvori radiane v stopinje (Hue)
            value = np.clip(amplitude / amplitude.max(), 0, 1)  # Normalizacija amplitude

            # Barvna kodacija v območju konture
            hsv_img[..., 0][mask == 255] = hue  # Hue
            hsv_img[..., 2][mask == 255] = value  # Value

        # Pretvorba iz HSV v RGB za prikaz
        rgb_img = cv2.cvtColor((hsv_img * 255).astype(np.uint8), cv2.COLOR_HSV2RGB)
        analize.append(rgb_img)

    return analize


In [29]:
dobljene_analize_synth_2_0 = analiziraj_opticni_pretok_2(slike_poti_synth_2)

Gibajoči elementi so bili uspešno povezani!


## Koda za obdelavo videoposnetka in pretvorbo v začetno zaporedje slik

In [ ]:
import cv2
import os

def video_to_frames(video_path, output_folder, resize_dim=None):
    # Ustvari mapo, če še ne obstaja
    os.makedirs(output_folder, exist_ok=True)

    # Odpri video
    video = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = video.read()
        if not ret:  # Ni več okvirjev za prebrati
            break

        # Če je določena dimenzija, spremeni velikost okvirja
        if resize_dim:
            frame = cv2.resize(frame, resize_dim)

        # Shrani okvir kot sliko
        frame_path = os.path.join(output_folder, f"frame_{frame_count:03d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    video.release()
    print(f"Video je razdeljen na {frame_count} slik, shranjenih v {output_folder}")

In [ ]:
# Pot do MP4 videa in mapo za shranjevanje slik
video_path = "ZaporedjeSlik/data/projekt_drugi_video.mp4"  # Zamenjaj z dejansko potjo do videa
output_folder = "ZaporedjeSlik/data/porocilo_drugi_video"  # Izberite izhodno mapo za slike

# Pokliči funkcijo z opcijsko spremembo velikosti okvirjev
video_to_frames(video_path, output_folder, resize_dim=(640, 480))